# importing embeddings, vocabulary, & functions file
We load the embeddings, vocabulary, and all the search functions. The functions have been predefined and are stored in the search-models subdirectory in the github repository, so we directly load them here.

In [2]:
import pandas as pd
import pickle
import json
import numpy as np
from search import SWOW
from pragmatics import RSA
from pragmatics import nonRSA
import walker 
%load_ext autoreload
%autoreload 2


# import walk data

In [3]:
import json
with open('../data/walk_data/intersection_candidates.json') as json_file:
    intersection_dict = json.load(json_file)
with open('../data/walk_data/union_candidates.json') as json_file:
    union_dict = json.load(json_file)
corrections = pd.read_csv('../data/corrections.csv')
vocab = pd.read_csv("../data/vocab.csv").rename(columns={"Word": "vocab_word"})
print(f"vocab is {len(vocab)} words")


vocab is 12218 words


# Experiment 1: candidate generation

In [45]:
generation_study_data = pd.read_csv('../data/exp1/e1_data.csv').melt(id_vars=['clueGiverID' , 'wordpair_id', 'Level', 'clueFinal'], value_vars=["clueOption1", "clueOption2", "clueOption3", "clueOption4", "clueOption5", "clueOption6", "clueOption7", "clueOption8"]).rename(columns={"value": "Clue1"})
generation_study_data = nonRSA.apply_corrections(generation_study_data, corrections, vocab)
generation_study_data = generation_study_data.dropna().drop_duplicates().groupby(['clueGiverID','wordpair_id',  'Level', 'clueFinal'], as_index=False)['correctedClue'].agg(','.join)
generation_study_data['clue_list'] = generation_study_data['value'].str.split(',')
display(generation_study_data)
common_candidates = nonRSA.get_common_candidates(union_dict, intersection_dict, generation_study_data, '../data/exp1/e1_common_candidates.csv')

,clueGiverID,wordpair_id,Level,clueFinal,value,clue_list
0,454955,bowl-sun,Medium,acai,"brunch,yard game,bright,acai","[brunch, yard game, bright, acai]"
1,454955,cave-knight,Hard,mysterious,"dark,dragon,unknown","[dark, dragon, unknown]"
2,454955,glow-quick,Medium,firefly,"candle,firefly,stick","[candle, firefly, stick]"
3,454955,gold-silver,Easy,jewelry,"jewelry,metal,expensive","[jewelry, metal, expensive]"
4,454955,oak-tree,Easy,forest,"bark,forest,wood","[bark, forest, wood]"
...,...,...,...,...,...,...
1451,491095,olive-real,Hard,non-GMO,"organic,garden,authentic,non-GMO","[organic, garden, authentic, non-GMO]"
1452,491095,regret-rude,Hard,bully,"bully,insult,fight","[bully, insult, fight]"
1453,491095,sit-stand,Easy,chair,"chair,attendance,opposites,stool","[chair, attendance, opposites, stool]"
1454,491095,stern-wind,Hard,sailing,"sailing,boat,gust","[sailing, boat, gust]"


# Experiment 2: original connector

In [33]:
e2_data = pd.read_csv("../data/exp2/e2_empirical_clues.csv", encoding= 'unicode_escape')
target_df = pd.read_csv("../data/targets.csv")
print(target_df.head())
representations = {}
representations['swow'] = pd.read_csv("../data/swow_associative_embeddings.csv").transpose().values
print(f"embeddings are shaped:", representations['swow'].shape)
with open('../data/exp2/e2_boards.json', 'r') as json_file:
    e2_boards = json.load(json_file)


     Word1     Word2 Experiment        boardnames   Level         wordpair
0     void     couch         E1   e1_board1_words    Hard       void-couch
1   giggle  abnormal         E1   e1_board1_words  Medium  giggle-abnormal
2     exam   algebra         E1   e1_board1_words    Easy     exam-algebra
3      tea      bean         E1  e1_board10_words    Easy         tea-bean
4  tourist    comedy         E1  e1_board10_words  Medium   tourist-comedy
embeddings are shaped: (12218, 300)


## non-RSA

Here we import the candidates from the json files and first compute how the non-RSA model would rank these candidates for each budget level. Next, we merge these obtained probabilities with the actual behavioral data 

In [5]:
board_optimal_params = {
    'swow' : (23.488850322875496, 1), # -13204
    'glove' : (20.952928531665275, 1), # -15774.814774380024)
    'bert-sum' : (19.983835225540847, 0.787924454045298),
}

In [34]:
e2_data = nonRSA.apply_corrections(e2_data, corrections, vocab)
e2_data['wordpair'] = e2_data['wordpair'].str.replace(' ', '')
e2_data = e2_data.merge(target_df, on='wordpair', how='left')
e2_data.to_csv("../data/exp2/e2_corrected.csv", index=False)
e2_data.head()


,Experiment_x,Board,Word1_x,Word2_x,Clue1,clueCount,wordpair,correctedClue,Word1_y,Word2_y,Experiment_y,boardnames,Level
0,E1,TrialList1,exam,algebra,calculus,1,exam-algebra,calculus,exam,algebra,E1,e1_board1_words,Easy
1,E1,TrialList1,exam,algebra,equation,1,exam-algebra,equation,exam,algebra,E1,e1_board1_words,Easy
2,E1,TrialList1,exam,algebra,knowledge,1,exam-algebra,knowledge,exam,algebra,E1,e1_board1_words,Easy
3,E1,TrialList1,exam,algebra,math,22,exam-algebra,math,exam,algebra,E1,e1_board1_words,Easy
4,E1,TrialList1,exam,algebra,school,2,exam-algebra,school,exam,algebra,E1,e1_board1_words,Easy


In [21]:
candidateprobs_nonRSA  = nonRSA.get_nonRSA_union_int(union_dict, intersection_dict, target_df, e2_boards, board_optimal_params, vocab, representations, e2_data, '../data/exp2/nonRSAprobs.csv')

## RSA

In [22]:
rsa_optimal_params = {
    'swow' : (25.1522030761838, 0.03863169001849234),
    'glove' : (82.83019661384789, 0.9997249702731884),
    'bert-sum' : (29.709602301411962, 0.031659060110267576), #-17533
}

In [25]:
candidateprobs_RSA  = RSA.get_RSA_union_int(union_dict, intersection_dict, target_df, e2_boards, rsa_optimal_params, vocab, representations, e2_data, '../data/exp2/RSAprobs.csv')

# Experiment 3: targeted endorsements
Here, we look at how the models predict specific target endorsements from the candidate set provided to the speaker

In [41]:
with open('../data/exp3/e3_boards.json', 'r') as json_file:
    e3_boards = json.load(json_file)
e3_stimuli = pd.read_csv('../data/exp3/e3_stimuli.csv')
## pass through corrections file
e3_stimuli = nonRSA.apply_corrections(e3_stimuli, corrections, vocab)
# create column that records whether correctedClue in vocab or not
e3_stimuli['correctedClue_in_vocab'] = e3_stimuli['correctedClue'].isin(vocab['vocab_word'])
# merge with target_df
e3_stimuli = e3_stimuli.merge(target_df, on='wordpair', how='left')
e3_stimuli.head()

,wordpair,accessibility,distinctiveness,Clue1,Word1_x,Word2_x,Board,Experiment_x,correctedClue,correctedClue_in_vocab,Word1_y,Word2_y,Experiment_y,boardnames,Level
0,exam-algebra,high,high,school,EXAM,ALGEBRA,TrialList1,E1,school,True,exam,algebra,E1,e1_board1_words,Easy
1,exam-algebra,low,low,calculus,EXAM,ALGEBRA,TrialList1,E1,calculus,True,exam,algebra,E1,e1_board1_words,Easy
2,exam-algebra,high,low,test,EXAM,ALGEBRA,TrialList1,E1,test,True,exam,algebra,E1,e1_board1_words,Easy
3,exam-algebra,low,high,math,EXAM,ALGEBRA,TrialList1,E1,math,True,exam,algebra,E1,e1_board1_words,Easy
4,giggle-abnormal,high,high,funny,GIGGLE,ABNORMAL,TrialList1,E1,funny,True,giggle,abnormal,E1,e1_board1_words,Medium


## non-RSA probabilities

In [44]:
e3_nonRSA = nonRSA.get_nonRSA_union_int(union_dict, intersection_dict, target_df, e3_boards, board_optimal_params, vocab, representations, e3_stimuli, '../data/exp3/nonRSAprobs.csv')

## RSA probabilities

In [96]:
e3_nonRSA  = RSA.get_RSA_union_int(union_dict, intersection_dict, target_df, e3_boards, rsa_optimal_params, vocab, representations, e3_stimuli, '../data/exp3/RSAprobs.csv')

# other code 

In [97]:
# code for finding closest matches to a given word (spell-check)
from english_words import english_words_set
'imitate' in english_words_set
import difflib
difflib.get_close_matches('turbine', list(english_words_set))


['turbine', 'turbinate', 'urine']

In [83]:
expdata_long = e3_stimuli.groupby(['wordpair'], as_index=False)['value'].agg(','.join)
expdata_long['clue_list'] = expdata_long['value'].str.split(',')
expdata_long = expdata_long.merge(target_df, on = "wordpair")
# count how many clues for each wordpair are not in vocab
expdata_long['len_cluelist_not_in_vocab'] = expdata_long['clue_list'].apply(lambda x: len([e for e in x if e not in list(vocab.vocab_word)]))
expdata_long_subset = expdata_long[expdata_long['len_cluelist_not_in_vocab'] == 0]


# Old: Dedicated functions for full vocab

In [11]:
## create boards and merge with expdata
combined_boards_df = pd.DataFrame(columns=['Experiment', 'Board','boardwords'])
combined_boards_df["Experiment"]  = ["E1"] * 10 + ["E2"] * 10
combined_boards_df["Board"] = ["TrialList" + str(i) for i in range(1,11)] * 2
combined_boards_df["boardnames"] = (['e1_board' + str(i) + '_words' for i in range(1,11)] 
                                  + ['e2_board' + str(i) + '_words' for i in range(1,11)])
combined_boards_df["boardwords"] = [boards[n] for n in combined_boards_df["boardnames"]]

expdata_new = pd.merge(expdata,combined_boards_df,on=['Board', 'Experiment'],how='left')
expdata_new["wordpair"] = expdata_new["Word1"] + "-" + expdata_new["Word2"]
board_combos = {board_name : search_funcs.RSA.compute_board_combos(board_name, boards) for board_name in boards.keys()}

## Non - RSA method


In [12]:
board_optimal_params = {
    'swow' : (23.488850322875496, 1), # -13204
    'glove' : (20.952928531665275, 1), # -15774.814774380024)
    'bert-sum' : (19.983835225540847, 0.787924454045298),
}

In [13]:
cluescoredf = nonRSA.speaker_targetboard_cluescores(['swow', 'glove'], board_optimal_params, board_combos, boards, list(vocab.vocab_word), vocab, representations, target_df, expdata_new)
cluescoredf.head()

for swow and alpha 0.0


KeyboardInterrupt: 

## RSA Method

In [ ]:
rsa_optimal_params = {
    'swow' : (25.1522030761838, 0.03863169001849234),
    'glove' : (22.336514544537227, 0.039),
    'bert-sum' : (29.709602301411962, 0.031659060110267576), #-17533
}

In [ ]:
pragmaticspeakerdf = search_funcs.RSA.get_speaker_df(representations, combined_boards_df,rsa_optimal_params, list(vocab.vocab_word), vocab, expdata_new, board_combos, target_df, boards)
pragmaticspeakerdf.head()